# Sound classification with YAMNet

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
import pandas as pd
from scipy import signal

In [10]:
# Load the model
model = hub.load("https://tfhub.dev/google/yamnet/1")
model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x18e4cbb2e00>

In [3]:
class_map_path = model.class_map_path()
class_map_path

<tf.Tensor: shape=(), dtype=string, numpy=b'C:\\Users\\hshla\\AppData\\Local\\Temp\\tfhub_modules\\9616fd04ec2360621642ef9455b84f4b668e219e\\assets\\yamnet_class_map.csv'>

In [4]:
pd.read_csv("C:\\Users\\hshla\\AppData\\Local\\Temp\\tfhub_modules\\9616fd04ec2360621642ef9455b84f4b668e219e\\assets\\yamnet_class_map.csv")

,index,mid,display_name
0,0,/m/09x0r,Speech
1,1,/m/0ytgt,"Child speech, kid speaking"
2,2,/m/01h8n0,Conversation
3,3,/m/02qldy,"Narration, monologue"
4,4,/m/0261r1,Babbling
...,...,...,...
516,516,/m/07p_0gm,Throbbing
517,517,/m/01jwx6,Vibration
518,518,/m/07c52,Television
519,519,/m/06bz3,Radio


다운받은 yamnet의 csv파일에 위와 같은 내용이 들어 있습니다.
아래의 def문을 활용해 키값중에 display_name를 가져오겠습니다.

In [5]:
# Find the name of the class with the top when mean-aggregated across frame.
def class_names_from_csv(class_map_csv_text):
    class_names = []
    with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            class_names.append(row["display_name"])
    return class_names

- pandas 데이터프레임으로 파일을 로드해서 확인해보았습니다. 모델 안에 여러가지 파일들이 내장되어 있습니다.
- 로드된 오디오가 적절한 sample_rate(16k)인지 확인하고 변환하는 메서드를 추가합니다. 그렇지 않으면 모델의 결과에 영향을 줄 수 있습니다.

In [6]:
class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)
#class_names 

In [11]:
def ensure_sample_rate(original_sample_rate, waveform,
                      desired_sample_rate = 16000):
    """Resample waveform if required"""
    if original_sample_rate !=desired_sample_rate:
        desired_length = int(round(float(len(waveform)) /
                                   original_sample_rate *desired_sample_rate))
        waveform = scipy.signal.resample(waveform, desired_length)
    return desired_sample_rate, waveform

# 사운드 파일 다운로드 및 준비
- 저는 음성파일이 있어서 음성파일을 가지고 사용했습니다 
- 16KHZ의 샘플레이트에서 모노wav파일이어야 합니다.

In [23]:
# wav 파일 로드
wav_file_name = "D:/13827.1.wav"
sample_rate, wav_data = wavfile.read(wav_file_name, "rb")
sample_rate, wav_data = ensure_sample_rate(sample_rate,wav_data)


In [22]:
wav_file_name = 'D:/13827.1.wav'
sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')
sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)

# Show some basic information about the audio.
duration = len(wav_data)/sample_rate
print(f'Sample rate: {sample_rate} Hz')
print(f'Total duration: {duration:.2f}s')
print(f'Size of the input: {len(wav_data)}')

# Listening to the wav file.
Audio(wav_data, rate=sample_rate)

Sample rate: 16000 Hz
Total duration: 5.57s
Size of the input: 89088


In [25]:
# 스케일링 수행 0~1사이의 값으로 변경한다.
waveform=wav_data/tf.int16.max 

In [26]:
waveform

array([ 0.00177696, -0.00039652,  0.00019444, ...,  0.00845282,
        0.0070493 ,  0.00694774])